In [1]:
import numpy as np
from numpy.matlib import repmat
import scipy.stats
import matplotlib.pyplot as plt
from ipywidgets import interact
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from io import StringIO
from scipy import special
from scipy.optimize import minimize
from __future__ import division

class ExtData:
    def __init__(self):
        self.etaE = None
        self.etaI = None
        self.fE   = 0.0
        self.fI   = 0.0
        # ON-OFF visual pathway
        self.etaON  = None
        self.fON    = 0.0
        self.etaOFF = None
        self.fOFF   = 0.0

class StatData:
    def __init__(self):
        self.vbarE = None
        self.vbarI = None
        self.wbarE = None
        self.wbarI = None
        self.vbar3E = None
        self.vbar3I = None
        self.vbar4E = None
        self.vbar4I = None
        
        self.VEs    = None
        self.VIs    = None
        self.DE     = None
        self.DI     = None


class RecData:
    def __init__(self):
        self.mE = None
        self.mI = None
        self.SEE = 0.0
        self.SEI = 0.0
        self.SIE = 0.0
        self.SII = 0.0
        # LR connections
        self.LEE  = 0.0
        self.LIE  = 0.0
        self.LEEF = 0.0
        self.LIEF = 0.0
        
        self.In   = None
        
class NetParams:
    def __init__(self):
        self.NHYP = 0
        self.NE   = 0
        self.NI   = 0
        # SETTING
        self.gL   = 0.0
        self.vL   = 0.0
        self.V    = None
        self.h    = 0.0
        self.N_divide = 0
        self.dt = 0.0
        self.Final_time = 0.0
        self.tauN  = 0.0

        
def VQs(Ext,Rec,NetParams):
    # EXTRACT FR 
    (etaE,etaI,fE,fI,etaON,etaOFF,fON,fOFF) = (Ext.etaE,Ext.etaI,Ext.fE,Ext.fI,Ext.etaON,Ext.etaOFF,Ext.fON,Ext.fOFF)
    (mE,mI,In,SEE,SEI,SIE,SII,LEE,LIE,LEEF,LIEF) = (Rec.mE,Rec.mI,Rec.In,Rec.SEE,Rec.SEI,Rec.SIE,
                                                    Rec.SII,Rec.LEE,Rec.LIE,Rec.LEEF,Rec.LIEF)
    (vL,gL,dt,tauN,NHYP) = (NetParams.vL,NetParams.gL,NetParams.dt,NetParams.tauN,NetParams.NHYP)
    # CALCULATE $i_{NMDA}(lT+T)$
    INMDA = np.zeros_like(In)
    for i in range(NHYP):
        # Exc<-Exc
        INMDA[i,0] = np.sum(LEE*mE*(1-np.exp(-dt/tauN))) - LEE*mE[i,0]*(1-np.exp(-dt/tauN))
        INMDA[i,1] = np.sum(LIE*mE*(1-np.exp(-dt/tauN))) - LIE*mE[i,0]*(1-np.exp(-dt/tauN))
    In = In*np.exp(-dt/tauN) + INMDA
    """
    VEs = (vL*gL+etaE*fE+SEE*mE-SEI*mI)/gL  # DO NOT TAKE LR INTO CONSIDERATION
    VIs = (vL*gL+etaI*fI+SIE*mE-SII*mI)/gL
    """
    
    VEs = (vL*gL+etaE*fE+SEE*mE-SEI*mI+(In[:,0]))/gL  # DO NOT TAKE LR INTO CONSIDERATION
    VIs = (vL*gL+etaI*fI+SIE*mE-SII*mI+(In[:,1]))/gL
           
    return (VEs,VIs,In)

def DQs(Ext,Rec,NetParams):
    # EXTRACT FR 
    (etaE,etaI,fE,fI,etaON,etaOFF,fON,fOFF) = (Ext.etaE,Ext.etaI,Ext.fE,Ext.fI,Ext.etaON,Ext.etaOFF,Ext.fON,Ext.fOFF)
    (mE,mI,In,SEE,SEI,SIE,SII,LEE,LIE,LEEF,LIEF) = (Rec.mE,Rec.mI,Rec.In,Rec.SEE,Rec.SEI,Rec.SIE,
                                                    Rec.SII,Rec.LEE,Rec.LIE,Rec.LEEF,Rec.LIEF)
    (vL,gL,dt,tauN,NE,NI) = (NetParams.vL,NetParams.gL,NetParams.dt,NetParams.tauN,NetParams.NE,NetParams.NI)
    # IGNORE $\sigma i_N$
    DE = (etaE*np.square(fE)+mE*np.square(SEE)/NE+mI*np.square(SEI)/NI)/gL
    DI = (etaI*np.square(fI)+mE*np.square(SIE)/NE+mI*np.square(SII)/NI)/gL
    return (DE,DI)

def rho_EQ(Vs,D,V):
    
    Rv = np.copy(V)
    (vT,vR) = (1.0,0.0)
    tmpg = np.greater(V,vR)
    indp = (np.where(tmpg))
    sqrtD  = np.sqrt(D)
    intovT  = special.dawsn((vT-Vs)/sqrtD)*np.exp(np.square(vT-Vs)/D)
    intovSD = special.dawsn(-Vs/sqrtD)*np.exp(np.square(Vs)/D)

    # compute R with V>vR case:
    Rv[indp] = -special.dawsn((V[indp]-Vs)/sqrtD)+np.exp(-np.square(V[indp]-Vs)/D)*intovT
    if(indp[0][0]>1):
        Rv[0:indp[0][0]] = np.exp(-np.square(V[0:indp[0][0]]-Vs)/D)*(-intovSD + intovT)
    
    tmpl = np.less(V,-2.0/3.0)
    indp = np.where(tmpl)
    Rv[indp] = 0.0
    sum_c = (V[2]-V[1])*np.sum(Rv)
    # print 'sum_c',sum_c
    Rv = Rv/sum_c
    
    return (Rv,sum_c)

def solveVbarWbar4(NetParams,StatV,Rec):
    # EXTRACT STATE VARIABLES
    (dt,gL) = (NetParams.dt,NetParams.gL)
    (vbarE,wbarE,vbar3E,vbar4E) = (StatV.vbarE,StatV.wbarE,StatV.vbar3E,StatV.vbar4E)
    (vbarI,wbarI,vbar3I,vbar4I) = (StatV.vbarI,StatV.wbarI,StatV.vbar3I,StatV.vbar4I)
    (VEs,VIs,DE,DI) = (StatV.VEs,StatV.VIs,StatV.DE,StatV.DI)
    (mE,mI) = (Rec.mE,Rec.mI)
    print 'mE,mI',mE,mI
    
    dtgL = dt*gL
    vbarE1 = vbarE + dtgL*(- mE/gL - (vbarE - VEs));
    vbarI1 = vbarI + dtgL*(- mI/gL - (vbarI - VIs));
    

    wbarE1 = wbarE + dtgL*(- mE/gL - 2.0*(wbarE - VEs*vbarE - 0.5*DE));
    wbarI1 = wbarI + dtgL*(- mI/gL - 2.0*(wbarI - VIs*vbarI - 0.5*DI));
    # print 'Wbar',wbarE,wbarI

    vbar3E1 = vbar3E + dtgL*(- mE/gL - 3.0*(vbar3E - VEs*wbarE - DE*vbarE));
    vbar3I1 = vbar3I + dtgL*(- mI/gL - 3.0*(vbar3I - VIs*wbarI - DI*vbarI));

    vbar4E1 = vbar4E + dtgL*(- mE/gL - 4.0*(vbar4E - VEs*vbar3E - 1.5*DE*wbarE));
    vbar4I1 = vbar4I + dtgL*(- mI/gL - 4.0*(vbar4I - VIs*vbar3I - 1.5*DI*wbarI));
    return (vbarE1,vbarI1,wbarE1,wbarI1,vbar3E1,vbar3I1,vbar4E1,vbar4I1)

def optfun(lambda_u,mu,x,Pq,fin,gamma):
    lambda_u = lambda_u[:]
    k  = np.size(mu)
    # mu = np.reshape(mu,[k,1])
    tt = np.zeros(k+1)
    tt[0] = 1
    tt[1:k+1]  = mu[:]
    # print 'mu: ',tt
    dx = x[1]-x[0]
    # print 'dx: ',dx,'lambda: ',lambda_u
    # print 'lambda: ', lambda_u
    # lambda_u = lambda0[:]
    N  =np.size(lambda_u)
    # print N,np.shape(fin)
    
    p  = Pq*np.exp(np.dot(fin[:,0:N],lambda_u))
    f  = dx*np.sum(p)-np.dot(np.reshape(tt,[1,k+1]),lambda_u)   
    # print 'f: ',f
    return f
    
    



  


In [2]:
Ext = ExtData()
Rec = RecData()
Net = NetParams()
StatV = StatData()
# NETWORK STRUCTURE
(NE,NI,NHYP) = (100,100,1) #(300,100,3)
(vL,gL,V_start,V_end,N_divide) = (0.0,0.05,-1.0,1.0,2000)
(dt,tauN,Final_time,step) = (0.1,80.0,350.0,0)
# connectivity matrix for individual hyper-column
(SEE,SEI,SIE,SII) = (0.369,0.294,0.294,0.369)#(0.16,0.46,0.52,0.24)
(LEE,LIE)         = (0.0,0.0)
# External and LGN feedforward,fast input
(fE,etaE,fI,etaI) = (0.028,1.60,0.028,1.75)#(0.0135,3.8,0.0132,3.5)
(fON,etaON,fOFF,etaOFF) = (0,0,0,0)#(0.0135,3.8,0.0132,3.5)
# Vbin
V = np.linspace(V_start,V_end,N_divide)
V = np.transpose(V)
h = V[1]-V[0]

# NETWORK STRUCTURE
(Net.NE,Net.NI,Net.NHYP) = (NE,NI,NHYP)
(Net.vL,Net.gL,Net.V,Net.h,Net.N_divide,Net.dt,Net.Final_time,Net.tauN) = (vL,gL,V,h,N_divide,dt,Final_time,tauN)
# connectivity matrix for individual hyper-column
(Rec.SEE,Rec.SEI,Rec.SIE,Rec.SII) = (SEE,SEI,SIE,SII)
(Rec.LEE,Rec.LIE)         = (LEE,LIE)
# External and LGN feedforward,fast input
(Ext.fE,Ext.etaE,Ext.fI,Ext.etaI) = (fE,etaE,fI,etaI)
(Ext.fON,Ext.etaON,Ext.fOFF,Ext.etaOFF) = (fE,etaE,fI,etaI)
# PUT TOGETHER


# New narray
mE = np.zeros([NHYP,1])
mI = np.zeros_like(mE)
In = np.zeros([NHYP,2]) # Inh<-Exc or Exc<-Exc
# PUT TOGETHER
Rec.mE = mE
Rec.mI = mI
Rec.In = In
gammaE = np.zeros([NHYP,5])
gammaI = np.zeros_like(gammaE)

# statistical narray
(PEq,PIq)   = (np.zeros([N_divide,NHYP]),np.zeros([N_divide,NHYP]))
(sumE,sumI) = (np.zeros(NHYP),np.zeros(NHYP))
(fiE,fiI)   = (np.zeros([5,NHYP]),np.zeros([5,NHYP]))
(F,FI)      = (np.zeros([2,NHYP]),np.zeros([2,NHYP]))
(La0,LaI0)  = (np.zeros([3,NHYP]),np.zeros([3,NHYP]))
(La1,LaI1)  = (np.zeros([3,NHYP]),np.zeros([3,NHYP]))
(RvE,RvI)  = (np.zeros([N_divide,NHYP]),np.zeros([N_divide,NHYP]))
fin         = np.zeros([NHYP,N_divide,3])   # moment2
counter,t,Max_iteration = 1,0,1



In [3]:
# start, initial state
var1 = pow((5/3/250.0),2)
source = np.exp(-np.square(V-0.0)/var1/2.0)/np.sqrt(2.0*np.pi*var1)
source = source/(h*np.sum(source))
(vbarE,wbarE,vbar3E,vbar4E) = (h*np.sum(V*source),h*(np.sum(np.square(V)*source)),
h*np.sum(np.power(V,3.0)*source),h*np.sum(np.power(V,4.0)*source))
(vbarE,wbarE,vbar3E,vbar4E) = (repmat(vbarE,NHYP,1),repmat(wbarE,NHYP,1),repmat(vbar3E,NHYP,1),
                               repmat(vbar4E,NHYP,1))

(vbarI,wbarI,vbar3I,vbar4I) = (vbarE,wbarE,vbar3E,vbar4E)
       
(VEs,VIs,In) = VQs(Ext,Rec,Net)
(DE,DI)      = DQs(Ext,Rec,Net)
# save state variables
(StatV.VEs,StatV.VIs,StatV.DE,StatV.DI) = (VEs,VIs,DE,DI)
(StatV.vbarE,StatV.wbarE,StatV.vbar3E,StatV.vbar4E) = (vbarE,wbarE,vbar3E,vbar4E)
(StatV.vbarI,StatV.wbarI,StatV.vbar3I,StatV.vbar4I) = (vbarI,wbarI,vbar3I,vbar4I)
# testing and printing
print VEs,VIs,DE,DI
# plt.figure()
for i in range(NHYP):
    (PEq[:,i],sumE[i]) = rho_EQ(VEs[i],DE[i],V)
    (PIq[:,i],sumI[i]) = rho_EQ(VIs[i],DI[i],V)
    gammaE[i,:] = [1,vbarE[i],wbarE[i],vbar3E[i],vbar4E[i]]
    gammaI[i,:] = [1,vbarI[i],wbarI[i],vbar3I[i],vbar4I[i]]
    fiE[:,i] = np.transpose(gammaE[i,:])
    fiI[:,i] = np.transpose(gammaI[i,:])
    moment2 = 1
    if moment2:
        F[:,i]  = fiE[1:3,i]
        FI[:,i] = fiI[1:3,i]
        La0[:,i]= np.transpose(gammaE[i,0:3])
        LaI0[:,i] = np.transpose(gammaI[i,0:3])
    N = np.size(np.squeeze(La0[:,i]))
    fin[i,:,0] = 1
    # check!
    Ncheck = np.size(np.squeeze(fin[i,0,:]))
    
    for n in range(1,N):
        fin[i,:,n] = V*fin[i,:,n-1]
    # plt.plot(np.squeeze(PEq[i,:]))
# plt.show()



[[ 0.896]] [[ 0.98]] [[ 0.025088]] [[ 0.02744]]


In [4]:
print wbarE

[[  4.44444444e-05]]


In [5]:
# INITIATE STARTING STATE
times = 10000
(sum_mE,sum_mI)  = (0.0,0.0)
counter_firing_step = 0
counter = 1
t = 0.0
while (t< Final_time):
    (VEs,VIs,In) = VQs(Ext,Rec,Net)
    (DE,DI)      = DQs(Ext,Rec,Net) 
    # save state variables
    (StatV.VEs,StatV.VIs,StatV.DE,StatV.DI) = (VEs,VIs,DE,DI)
    print 'VEs: ',StatV.VEs,'VIs: ',StatV.VIs,'DE: ',StatV.DE,'DI: ',StatV.DI
    [vbarE,vbarI,wbarE,wbarI,vbar3E,vbar3I,vbar4E,vbar4I] = solveVbarWbar4(Net,StatV,Rec)
    # print 'mu and D',vbarE,vbarI,wbarE,wbarI,vbar3E,vbar3I,vbar4E,vbar4I

    
    (StatV.vbarE,StatV.wbarE,StatV.vbar3E,StatV.vbar4E) = (vbarE,wbarE,vbar3E,vbar4E)
    (StatV.vbarI,StatV.wbarI,StatV.vbar3I,StatV.vbar4I) = (vbarI,wbarI,vbar3I,vbar4I)
    
    for i in range(NHYP):
        (PEq[:,i],sumE[i]) = rho_EQ(VEs[i],DE[i],V)
        (PIq[:,i],sumI[i]) = rho_EQ(VIs[i],DI[i],V)
        gammaE[i,:] = [1,vbarE[i],wbarE[i],vbar3E[i],vbar4E[i]]
        gammaI[i,:] = [1,vbarI[i],wbarI[i],vbar3I[i],vbar4I[i]]
        fiE[:,i] = np.transpose(gammaE[i,:])
        fiI[:,i] = np.transpose(gammaI[i,:])
        # print 'fiE: ',fiE[:,i],'fiI: ',fiI[:,i]
        
        moment2 = 1
        if moment2:
            F[:,i]  = fiE[1:3,i]
            FI[:,i] = fiI[1:3,i]
            # def optfun(lambda_u,mu,x,PEq,fin,gamma):
            (tmu,tx,tPEq,tfin,tgamma) = (F[:,i],V,PEq[:,i],fin[i,:,:],1)
            # N-order,1),Vbin,1),Vbin,1),Vbin,N-order),
            a0 = La0[:,i]
            # print 'F/FI: ',F[:,i],FI[:,i]
            # print 'params: ',a0,tmu,tgamma
            res = minimize(optfun,a0,args=(tmu,tx,tPEq,tfin,tgamma),options={'disp': False})
            # print res
            La1[:,i] = res.x
            # print res.x
            (tmu,tx,tPIq,tfin,tgamma) = (FI[:,i],V,PIq[:,i],fin[i,:,:],1)  
            a0 = LaI0[:,i]
            res = minimize(optfun,a0,args=(tmu,tx,tPIq,tfin,tgamma),options={'disp': False})
            LaI1[:,i] = res.x
            La0[:,i] = np.real(La1[:,i])
            LaI0[:,i]= np.real(LaI1[:,i])
            
            RvE[:,i] = PEq[:,i]*np.exp(np.dot(np.squeeze(fin[i,:,:]),La1[:,i]))
            RvI[:,i] = PIq[:,i]*np.exp(np.dot(np.squeeze(fin[i,:,:]),LaI1[:,i]))
            # normalization
            RvE[:,i] = RvE[:,i]/((V[1]-V[0])*np.sum(RvE[:,i]))
            RvI[:,i] = RvI[:,i]/((V[1]-V[0])*np.sum(RvI[:,i]))
            mE[i] = gL*np.sqrt(DE[i])*np.exp(np.sum(La1[:,i]))/sumE[i]/2
            mI[i] = gL*np.sqrt(DI[i])*np.exp(np.sum(LaI1[:,i]))/sumI[i]/2
            
    # PUT TOGETHER
    Rec.mE = mE
    Rec.mI = mI
    Rec.In = In
    print 'mE: ',Rec.mE,' mI: ',Rec.mI,' iNMDA: ',Rec.In
    t = dt*counter
    counter = counter + 1
    step    = step + 1
    """
    if step ==5:
        plt.plot(V,RvE[:,0])
        plt.show()
        step = 0
    """    
    

VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[ 0.]] [[ 0.]]
mE:  [[ 0.]]  mI:  [[ 0.]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[ 0.]] [[ 0.]]
mE:  [[ 0.]]  mI:  [[ 0.]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[ 0.]] [[ 0.]]
mE:  [[ 0.]]  mI:  [[ 0.]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[ 0.]] [[ 0.]]
mE:  [[  4.82523863e-285]]  mI:  [[  2.29864106e-269]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  4.82523863e-285]] [[  2.29864106e-269]]
mE:  [[  1.93548065e-235]]  mI:  [[  1.04993160e-223]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  1.93548065e-235]] [[  1.04993160e-223]]
mE:  [[  2.07561804e-203]]  mI:  [[  4.37714523e-194]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs

VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  3.04249064e-37]] [[  2.65953404e-33]]
mE:  [[  2.22321829e-36]]  mI:  [[  1.72794986e-32]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  2.22321829e-36]] [[  1.72794986e-32]]
mE:  [[  1.50588876e-35]]  mI:  [[  1.04345155e-31]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  1.50588876e-35]] [[  1.04345155e-31]]
mE:  [[  9.62488098e-35]]  mI:  [[  5.88130028e-31]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  9.62488098e-35]] [[  5.88130028e-31]]
mE:  [[  5.61338246e-34]]  mI:  [[  3.08933509e-30]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  5.61338246e-34]] [[  3.08933509e-30]]
mE:  [[  3.05641961e-33]]  mI:  [[  1.53226357e-29]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[

VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  3.77771009e-17]] [[  1.41141496e-14]]
mE:  [[  6.15253010e-17]]  mI:  [[  2.22030234e-14]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  6.15253010e-17]] [[  2.22030234e-14]]
mE:  [[  9.89308812e-17]]  mI:  [[  3.45680813e-14]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  9.89308812e-17]] [[  3.45680813e-14]]
mE:  [[  1.57334362e-16]]  mI:  [[  5.31804775e-14]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  1.57334362e-16]] [[  5.31804775e-14]]
mE:  [[  2.47384558e-16]]  mI:  [[  8.11040032e-14]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  2.47384558e-16]] [[  8.11040032e-14]]
mE:  [[  3.84123664e-16]]  mI:  [[  1.22338404e-13]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.896]] VIs:  [[ 0.98]] DE:  [[

mE:  [[  1.93826213e-10]]  mI:  [[  2.72980392e-08]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89599984]] VIs:  [[ 0.9799998]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  1.93826213e-10]] [[  2.72980392e-08]]
mE:  [[  2.29305585e-10]]  mI:  [[  3.18411062e-08]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89599981]] VIs:  [[ 0.97999977]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  2.29305585e-10]] [[  3.18411062e-08]]
mE:  [[  2.70788725e-10]]  mI:  [[  3.70452973e-08]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89599978]] VIs:  [[ 0.97999973]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  2.70788725e-10]] [[  3.70452973e-08]]
mE:  [[  3.19105239e-10]]  mI:  [[  4.30047697e-08]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89599975]] VIs:  [[ 0.97999968]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  3.19105239e-10]] [[  4.30047697e-08]]
mE:  [[  3.89807474e-10]]  mI:  [[  5.14202543e-08]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.8959997]] VIs:  [[ 0.97999962]] DE:  [[ 0.025088]] DI:  [[ 0.02744]]
mE,mI [[  3.89807474e-10]] [[  5.14

mE:  [[  6.86659825e-08]]  mI:  [[  5.61023170e-06]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89596752]] VIs:  [[ 0.979959]] DE:  [[ 0.0250881]] DI:  [[ 0.02744015]]
mE,mI [[  6.86659825e-08]] [[  5.61023170e-06]]
mE:  [[  7.79370796e-08]]  mI:  [[  6.12174171e-06]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89596458]] VIs:  [[ 0.97995528]] DE:  [[ 0.02508811]] DI:  [[ 0.02744017]]
mE,mI [[  7.79370796e-08]] [[  6.12174171e-06]]
mE:  [[  8.60338947e-08]]  mI:  [[  6.84215633e-06]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.8959604]] VIs:  [[ 0.97995001]] DE:  [[ 0.02508812]] DI:  [[ 0.02744019]]
mE,mI [[  8.60338947e-08]] [[  6.84215633e-06]]
mE:  [[  9.75955278e-08]]  mI:  [[  7.45004341e-06]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89595691]] VIs:  [[ 0.97994559]] DE:  [[ 0.02508813]] DI:  [[ 0.0274402]]
mE,mI [[  9.75955278e-08]] [[  7.45004341e-06]]
mE:  [[  1.07499631e-07]]  mI:  [[  8.30862975e-06]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89595194]] VIs:  [[ 0.97993931]] DE:  [[ 0.02508815]] DI:  [[ 0.02744023]]
mE,mI [[  1.0

...

mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  

mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  

mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  

mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  

VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  [[ 0.028012]]
mE,mI [[ 0.01078085]] [[ 0.01416074]]
mE:  [[ 0.01078085]]  mI:  [[ 0.01416074]]  iNMDA:  [[ 0.  0.]]
VEs:  [[ 0.89229747]] VIs:  [[ 0.93888509]] DE:  [[ 0.02562639]] DI:  

In [ ]:
def Solve_moment_Python_CODE_method():
    Ext = ExtData()
    Rec = RecData()
    Net = NetParams()
    StatV = StatData()
    # NETWORK STRUCTURE
    (NE,NI,NHYP) = (100,100,1) #(300,100,3)
    (vL,gL,V_start,V_end,N_divide) = (0.0,0.05,-1.0,1.0,2000)
    (dt,tauN,Final_time,step) = (0.1,80.0,350.0,0)
    # connectivity matrix for individual hyper-column
    (SEE,SEI,SIE,SII) = (0.369,0.294,0.294,0.369)#(0.16,0.46,0.52,0.24)
    (LEE,LIE)         = (0.0,0.0)
    # External and LGN feedforward,fast input
    (fE,etaE,fI,etaI) = (0.028,1.76,0.028,1.76)#(0.0135,3.8,0.0132,3.5)
    (fON,etaON,fOFF,etaOFF) = (0,0,0,0)#(0.0135,3.8,0.0132,3.5)
    # Vbin
    V = np.linspace(V_start,V_end,N_divide)
    V = np.transpose(V)
    h = V[1]-V[0]
    
    # NETWORK STRUCTURE
    (Net.NE,Net.NI,Net.NHYP) = (NE,NI,NHYP)
    (Net.vL,Net.gL,Net.V,Net.h,Net.N_divide,Net.dt,Net.Final_time,Net.tauN) = (vL,gL,V,h,N_divide,dt,Final_time,tauN)
    # connectivity matrix for individual hyper-column
    (Rec.SEE,Rec.SEI,Rec.SIE,Rec.SII) = (SEE,SEI,SIE,SII)
    (Rec.LEE,Rec.LIE)         = (LEE,LIE)
    # External and LGN feedforward,fast input
    (Ext.fE,Ext.etaE,Ext.fI,Ext.etaI) = (fE,etaE,fI,etaI)
    (Ext.fON,Ext.etaON,Ext.fOFF,Ext.etaOFF) = (fE,etaE,fI,etaI)
    # PUT TOGETHER
    
    
    # New narray
    mE = np.zeros([NHYP,1])
    mI = np.zeros_like(mE)
    In = np.zeros([NHYP,2]) # Inh<-Exc or Exc<-Exc
    # PUT TOGETHER
    Rec.mE = mE
    Rec.mI = mI
    Rec.In = In
    gammaE = np.zeros([NHYP,5])
    gammaI = np.zeros_like(gammaE)
    
    # statistical narray
    (PEq,PIq)   = (np.zeros([N_divide,NHYP]),np.zeros([N_divide,NHYP]))
    (sumE,sumI) = (np.zeros(NHYP),np.zeros(NHYP))
    (fiE,fiI)   = (np.zeros([5,NHYP]),np.zeros([5,NHYP]))
    (F,FI)      = (np.zeros([2,NHYP]),np.zeros([2,NHYP]))
    (La0,LaI0)  = (np.zeros([3,NHYP]),np.zeros([3,NHYP]))
    (La1,LaI1)  = (np.zeros([3,NHYP]),np.zeros([3,NHYP]))
    (RvE,RvI)  = (np.zeros([N_divide,NHYP]),np.zeros([N_divide,NHYP]))
    fin         = np.zeros([NHYP,N_divide,3])   # moment2
    counter,t,Max_iteration = 1,0,1
    
    # start, initial state
    var1 = pow((5/3/250.0),2)
    source = np.exp(-np.square(V-0.0)/var1/2.0)/np.sqrt(2.0*np.pi*var1)
    source = source/(h*np.sum(source))
    (vbarE,wbarE,vbar3E,vbar4E) = (h*np.sum(V*source),h*(np.sum(np.square(V)*source)),
    h*np.sum(np.power(V,3.0)*source),h*np.sum(np.power(V,4.0)*source))
    (vbarE,wbarE,vbar3E,vbar4E) = (repmat(vbarE,NHYP,1),repmat(wbarE,NHYP,1),repmat(vbar3E,NHYP,1),
                                   repmat(vbar4E,NHYP,1))
    
    (vbarI,wbarI,vbar3I,vbar4I) = (vbarE,wbarE,vbar3E,vbar4E)
           
    (VEs,VIs,In) = VQs(Ext,Rec,Net)
    (DE,DI)      = DQs(Ext,Rec,Net)
    # save state variables
    (StatV.VEs,StatV.VIs,StatV.DE,StatV.DI) = (VEs,VIs,DE,DI)
    (StatV.vbarE,StatV.wbarE,StatV.vbar3E,StatV.vbar4E) = (vbarE,wbarE,vbar3E,vbar4E)
    (StatV.vbarI,StatV.wbarI,StatV.vbar3I,StatV.vbar4I) = (vbarI,wbarI,vbar3I,vbar4I)
    # testing and printing
    print VEs,VIs,DE,DI
    # plt.figure()
    for i in range(NHYP):
        (PEq[:,i],sumE[i]) = rho_EQ(VEs[i],DE[i],V)
        (PIq[:,i],sumI[i]) = rho_EQ(VIs[i],DI[i],V)
        gammaE[i,:] = [1,vbarE[i],wbarE[i],vbar3E[i],vbar4E[i]]
        gammaI[i,:] = [1,vbarI[i],wbarI[i],vbar3I[i],vbar4I[i]]
        fiE[:,i] = np.transpose(gammaE[i,:])
        fiI[:,i] = np.transpose(gammaI[i,:])
        moment2 = 1
        if moment2:
            F[:,i]  = fiE[1:3,i]
            FI[:,i] = fiI[1:3,i]
            La0[:,i]= np.transpose(gammaE[i,0:3])
            LaI0[:,i] = np.transpose(gammaI[i,0:3])
        N = np.size(np.squeeze(La0[:,i]))
        fin[i,:,0] = 1
        # check!
        Ncheck = np.size(np.squeeze(fin[i,0,:]))
        
        for n in range(1,N):
            fin[i,:,n] = V*fin[i,:,n-1]
        # plt.plot(np.squeeze(PEq[i,:]))
    # plt.show()
    
    # INITIATE STARTING STATE
    times = 10000
    (sum_mE,sum_mI)  = (0.0,0.0)
    counter_firing_step = 0
    counter = 1
    t = 0.0
    while (t<1.2):# Final_time):
        (VEs,VIs,In) = VQs(Ext,Rec,Net)
        (DE,DI)      = DQs(Ext,Rec,Net) 
        # save state variables
        (StatV.VEs,StatV.VIs,StatV.DE,StatV.DI) = (VEs,VIs,DE,DI)
        print 'VEs: ',StatV.VEs,'VIs: ',StatV.VIs,'DE: ',StatV.DE,'DI: ',StatV.DI
        [vbarE,vbarI,wbarE,wbarI,vbar3E,vbar3I,vbar4E,vbar4I] = solveVbarWbar4(Net,StatV,Rec)
        print 'mu and D',vbarE,vbarI,wbarE,wbarI,vbar3E,vbar3I,vbar4E,vbar4I

        
        (StatV.vbarE,StatV.wbarE,StatV.vbar3E,StatV.vbar4E) = (vbarE,wbarE,vbar3E,vbar4E)
        (StatV.vbarI,StatV.wbarI,StatV.vbar3I,StatV.vbar4I) = (vbarI,wbarI,vbar3I,vbar4I)
        
        for i in range(NHYP):
            (PEq[:,i],sumE[i]) = rho_EQ(VEs[i],DE[i],V)
            (PIq[:,i],sumI[i]) = rho_EQ(VIs[i],DI[i],V)
            gammaE[i,:] = [1,vbarE[i],wbarE[i],vbar3E[i],vbar4E[i]]
            gammaI[i,:] = [1,vbarI[i],wbarI[i],vbar3I[i],vbar4I[i]]
            fiE[:,i] = np.transpose(gammaE[i,:])
            fiI[:,i] = np.transpose(gammaI[i,:])
            # print 'fiE: ',fiE[:,i],'fiI: ',fiI[:,i]
            
            moment2 = 1
            if moment2:
                F[:,i]  = fiE[1:3,i]
                FI[:,i] = fiI[1:3,i]
                # def optfun(lambda_u,mu,x,PEq,fin,gamma):
                (tmu,tx,tPEq,tfin,tgamma) = (F[:,i],V,PEq[:,i],fin[i,:,:],1)
                # N-order,1),Vbin,1),Vbin,1),Vbin,N-order),
                a0 = La0[:,i]
                res = minimize(optfun,a0,args=(tmu,tx,tPEq,tfin,tgamma),method='Nelder-Mead',
                               options={'disp': False})
                # print res
                La1[:,i] = res.x
                (tmu,tx,tPEq,tfin,tgamma) = (FI[:,i],V,PIq[:,i],fin[i,:,:],1)  
                a0 = LaI0[:,i]
                res = minimize(optfun,a0,args=(tmu,tx,tPEq,tfin,tgamma),method='Nelder-Mead',
                               options={'disp': False})
                LaI1[:,i] = res.x
                La0[:,i] = np.real(La1[:,i])
                LaI0[:,i]= np.real(LaI1[:,i])
                
                RvE[:,i] = PEq[:,i]*np.exp(np.dot(np.squeeze(fin[i,:,:]),La1[:,i]))
                RvI[:,i] = PIq[:,i]*np.exp(np.dot(np.squeeze(fin[i,:,:]),LaI1[:,i]))
                # normalization
                RvE[:,i] = RvE[:,i]/((V[1]-V[0])*np.sum(RvE[:,i]))
                RvI[:,i] = RvI[:,i]/((V[1]-V[0])*np.sum(RvI[:,i]))
                mE[i] = gL*np.sqrt(DE[i])*np.exp(np.sum(La1[:,i]))/sumE[i]/2
                mI[i] = gL*np.sqrt(DI[i])*np.exp(np.sum(LaI1[:,i]))/sumI[i]/2
                
        # PUT TOGETHER
        Rec.mE = mE
        Rec.mI = mI
        Rec.In = In
        print 'mE: ',Rec.mE,' mI: ',Rec.mI,' iNMDA: ',Rec.In
        t = dt*counter
        counter = counter + 1
        step    = step + 1
            
        if step ==5:
            plt.plot(V,RvE[:,0])
            plt.show()
            step = 0
                
    
        
    
    # equalibrium
    
    return wbarE
    
Source = Solve_moment_Python_CODE_method()